Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


In [2]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

corpus length: 600901
total chars: 59
nb sequences: 200287
Vectorization...


#### LSTM

```python
keras.layers.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, implementation=1, return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False)
```

#### Long-Short Term Memory layer - Hochreiter 1997.

**Arguments**
* units: Positive integer, dimensionality of the output space.
* activation: Activation function to use (see activations). If you pass None, no activation is applied (ie. "linear" activation: a(x) = x).
* recurrent_activation: Activation function to use for the recurrent step (see activations).
* use_bias: Boolean, whether the layer uses a bias vector.
* kernel_initializer: Initializer for the kernel weights matrix, used for the linear transformation of the inputs. (see initializers).
* recurrent_initializer: Initializer for the recurrent_kernel weights matrix, used for the linear transformation of the recurrent state. (see initializers).
* bias_initializer: Initializer for the bias vector (see initializers).
* unit_forget_bias: Boolean. If True, add 1 to the bias of the forget gate at initialization. Setting it to true will also force bias_initializer="zeros". This is recommended in Jozefowicz et al.
* kernel_regularizer: Regularizer function applied to the kernel weights matrix (see regularizer).
* recurrent_regularizer: Regularizer function applied to the recurrent_kernel weights matrix (see regularizer).
bias_regularizer: Regularizer function applied to the bias vector (see regularizer).
activity_regularizer: Regularizer function applied to the output of the layer (its "activation"). (see regularizer).
kernel_constraint: Constraint function applied to the kernel weights matrix (see constraints).
recurrent_constraint: Constraint function applied to the recurrent_kernel weights matrix (see constraints).
bias_constraint: Constraint function applied to the bias vector (see constraints).
dropout: Float between 0 and 1. Fraction of the units to drop for the linear transformation of the inputs.
recurrent_dropout: Float between 0 and 1. Fraction of the units to drop for the linear transformation of the recurrent state.
implementation: Implementation mode, either 1 or 2. Mode 1 will structure its operations as a larger number of smaller dot products and additions, whereas mode 2 will batch them into fewer, larger operations. These modes will have different performance profiles on different hardware and for different applications.
return_sequences: Boolean. Whether to return the last output. in the output sequence, or the full sequence.
return_state: Boolean. Whether to return the last state in addition to the output.
go_backwards: Boolean (default False). If True, process the input sequence backwards and return the reversed sequence.
stateful: Boolean (default False). If True, the last state for each sample at index i in a batch will be used as initial state for the sample of index i in the following batch.
unroll: Boolean (default False). If True, the network will be unrolled, else a symbolic loop will be used. Unrolling can speed-up a RNN, although it tends to be more memory-intensive. Unrolling is only suitable for short sequences.

In [3]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [3]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [5]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
200287/200287 [==============================] - 76s - loss: 1.9757    

----- diversity: 0.2
----- Generating with seed: "actions?
in the end, however, though sil"
actions?
in the end, however, though sill the desire and and and and and the some of the subligin and and and and and and and and and and and and subligin that the subligin that whil in the same and and and and and and and and and and and and and and something and and and and the presention of the conserding and and and the sublignd the does of the presentive and and and and and and and and and and and and and and and and and and and an

----- diversity: 0.5
----- Generating with seed: "actions?
in the end, however, though sil"
actions?
in the end, however, though sillity of the preservess of the spirit of the sublignce, and and and and the stree and as a subligin in the sublection of man and a now the last the sence of the preserved in is not ald the

or ape, that attempts to pluck the fruit of the present and and and interpretations of the present spective of the suffering the stand the most conscience of the present of the strength in the standpoints of the present species of the present being the part of the state of the entire to the most power the streaded to the present of the standpoint of the conscience is the most person of the soul of the standpoints of the present stand th

----- diversity: 0.5
----- Generating with seed: "or ape, that attempts to pluck the fruit"
or ape, that attempts to pluck the fruit of the compored of the such a great to the case of the world to instance, as the way in the state of men of the art of the conscience is the present of the most spirit of the substance, and and intellicted to himself to the most constition and suspicious and the most friensty of the fores the most possibility of the most super-the subvalistance the standberation to whom in the higher the spirit o

----- diversity: 1.0
---

in order that the continuance of the experience and the same profound and in the spirit is the present and and believed that is the same and something in the same as the contempt the same an age to probably and and the same interpretation of the spirit is the probably any the taste of the spirit is a properness of the contempt of the same standpoint of the same sensive and of the spirit is the same interpretation of th

----- diversity: 0.5
----- Generating with seed: "ined distraction.
in order that the cont"
ined distraction.
in order that the conted and believed that it is not not be to only the one the act the the possible to himself all contempe of the order to his scholars and and the
insires as a soul. the contemph sand he is happice will not
even in the every animal from the true of the extent in the sonity of which so that they could to known the same viwing their new his father and extent and as the content faring himself and
in the

----- diversity: 1.0
----- Generating with

in many interpretation of the present and the state of the sense of the state of the above all and and soul and some distincting to the properable and soul and and soul and the common of the heart, and so the present and the most desire of a strong in the promited the present and more of the most world in the most sense--where we are not alone of the stated to the state of the state of the most sensat

----- diversity: 0.5
----- Generating with seed: " his morals that "what is estimable
in m"
 his morals that "what is estimable
in more of the other primited sense of the present different even the souls to the commons of morality--it will not the discertional of the fundamental into-nour and influence in the world is not do as if he possessible in the world in the surcess and havity of the conditionate and development of such as to himself to the uniroranist of the refined upon the case of men of the true nature, the unity in

----- diversity: 1.0
----- Generating with seed: " his moral

C:\Users\wszjz\AppData\Local\conda\conda\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


...

"possible of all the superstitious, and anarvessible f

----- diversity: 1.0
----- Generating with seed: "lves only the more refined forms of gove"
lves only the more refined forms of goveat,
circumstance, who can are and avirmpture-divinal impurpedly to men men on all age some "pain of
europe and cluenness of a dishe-can
leads for nature; in lover
to mother upon
his endiavists, and has are from anne witn much beform sample will as i am" produces that anocherously
eartic reface to all a relusion cannot was not etred" in adtinial of the very and standpoirs, any laigumes"; in revenge

----- diversity: 1.2
----- Generating with seed: "lves only the more refined forms of gove"
lves only the more refined forms of govers at a good and nages, what care greating, to the fellow, but as resire with "many to former--so far whether the who would finally if us.

phrerit geature in the speaks out-ibolers, puration,
in be absts any bology or plato, flaer",
and who endurace habits, thoughtast bl misha

hows herself as she is, in all her extraordinary in the such assume and such a superstition to the such as the proble to the subject of the such as the masterian and such a soul and conscience of the sense of the subject of the same impulsess of the superstition of the same impulses of the such as a subjection of the same impulsess of the same as the profoundly the such assume of the spirits, and in the most own and consideration of the

----- diversity: 0.5
----- Generating with seed: "hows herself as she is, in all her extra"
hows herself as she is, in all her extraordinary very prevailing fact that it was every his world of the properation
of the condition of the most state and such a free-sight of the lact of the price for the superstition in the stronger desires of the
considering than the possess, in the same future of all the fact that is a means of man counts the part of woman in a superstition are not to holy, the sense and to what is to the most
mean

----- diversity: 1.0
---

re of the task imposed upon us by our ages to the same as the promited to the same as the same as the concerning the sense of the sense of the sentiment of the truths of the sentiment of the concerning the spirits of the superstitions of the truths of the concerning the sense and such a scholars of the concerning the truths and self-same in the spiritual other and sentiments of the sense of the truths and and as the most sentiments of t

----- diversity: 0.5
----- Generating with seed: "re of the task imposed upon us by our ag"
re of the task imposed upon us by our age man and out of the every to the surpribers is the must considers of the concerning the other and man in the most delight of the things, and considers, and if but to be cause of the truth of the case the nature, the world of the mable the same in the most prevaigs in the entition certain and the long that if men of him
as a relations of the good eniminity in the "deference to the superstition of 

----- diversity: 1.0
---

among the species who distinguish of the sense of the conceive the more and some higher the sense of the conception of the trained to himself and in the conception the sense of the conceives the self-despect of the strength of the conception and sense of the powerful and self-sacrifice of the truth. the more and and as a powerful and what it is the sense of the conception that the most sentiment of the sense of the world of th

----- diversity: 0.5
----- Generating with seed: "xceptions
among the species who distingu"
xceptions
among the species who distinguishing morality of the happiness to the prederents and similar conceived. one who be old preserving the actions, as a receives.


gimate; and in his specied
would not be have all the confecture of the contempting as to whom the possibility of conceit of what actions and the words of the world in order to the world enough and perhaps they ever at the former morality of the sense and self-spain and 

----- diversity: 1.0
----- Generat

=discredited words.=--away with the desirable the process of the stand by the philosophers and we are the stand by the more and in the former and an interest the most spirits of the more interest the surprised to be for the protective and in the superiority of the stated the process of the works of the stand by the more and an interest and in the process the most spirits and the other in the more that the former and in the statesment

----- diversity: 0.5
----- Generating with seed: "8

=discredited words.=--away with the d"
8

=discredited words.=--away with the dispose and the we shall done been a responsibility are on a will to the promptive spirit being strength, and and the
experiences of a conception of such formerly by the entities is the cheers which has a souls
of the morality. the difficult and to only assentive believe that the dissick of the assertion has been for their fatue of contrary, which is the secret in the point of such desired to find 

----- diversity: 1.0
----- 